In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
#from psycopg2 import extras as ex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from sklearn.preprocessing import *
import datetime

## prepare data

In [2]:
# connect to the postgresql
db_connection = psycopg2.connect(host='127.0.0.1',dbname="chicago_business_intelligence", user="postgres" , password="12345")
cursor = db_connection.cursor()

In [3]:
# get data
select_data = (
    """
        SELECT * FROM "neighborhood_community_zip";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_1 = pd.DataFrame(result_1)
df_1.columns = ["id","geo_id","zipcode","community","neighborhood"]
df_1 = df_1[["zipcode","community","neighborhood"]]
df_1.zipcode = df_1.zipcode.astype("int")
df_1

,zipcode,community,neighborhood
0,60601,Loop,Printer's Row
1,60602,Loop,Printer's Row
2,60603,Loop,Printer's Row
3,60604,Loop,Printer's Row
4,60605,Loop,Printer's Row
...,...,...,...
306,60652,Ashburn,Parkview
307,60652,Ashburn,Scottsdale
308,60652,Ashburn,Wrightwood
309,60656,O'Hare,Schorsch Forest View


In [4]:
# get data
select_data = (
    """
        SELECT * FROM "building_permits";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_2 = pd.DataFrame(result_1)
df_2.columns = ["id","permit_id","permit_type","total_fee","latitude","longitude","zipcode"]
df_2 = df_2[["permit_id","permit_type","zipcode"]]
df_2.zipcode = df_2.zipcode.astype("int")
df_2

,permit_id,permit_type,zipcode
0,3232764,PERMIT - ELECTRIC WIRING,60631
1,3219030,PERMIT - ELECTRIC WIRING,60615
2,3248507,PERMIT - EASY PERMIT PROCESS,60618
3,3248478,PERMIT - EASY PERMIT PROCESS,60623
4,3248576,PERMIT - EASY PERMIT PROCESS,60629
...,...,...,...
993,3246709,PERMIT - ELECTRIC WIRING,60607
994,3246907,PERMIT - ELECTRIC WIRING,60647
995,3246877,PERMIT - ELECTRIC WIRING,60604
996,3246819,PERMIT - EASY PERMIT PROCESS,60647


In [5]:
# get data
select_data = (
    """
        SELECT * FROM "public_health_statistics";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_3 = pd.DataFrame(result_1)
df_3.columns = ["id","community_area_name","below_poverty_level","per_capita_income","unemployment"]
df_3 = df_3[["community_area_name","per_capita_income"]]
df_3.columns = ["community","per_capita_income"]
df_3

,community,per_capita_income
0,Rogers Park,23714.0
1,West Ridge,21375.0
2,Uptown,32355.0
3,Lincoln Square,35503.0
4,North Center,51615.0
...,...,...
72,Washington Heights,19709.0
73,Mount Greenwood,34221.0
74,Morgan Park,26185.0
75,O'Hare,29402.0


## process data

In [6]:
pro_df_1 = df_3.merge(df_1,how='inner', on='community')
pro_df_2 = pro_df_1.merge(df_2,how='inner', on='zipcode')
pro_df_2 = pro_df_2[["permit_id","permit_type","per_capita_income","community"]]
pro_df_2 = pro_df_2.reset_index(drop=True)
pro_df_2

,permit_id,permit_type,per_capita_income,community
0,3248257,PERMIT - EASY PERMIT PROCESS,23714.0,Rogers Park
1,3247837,PERMIT - EASY PERMIT PROCESS,23714.0,Rogers Park
2,3232986,PERMIT - RENOVATION/ALTERATION,23714.0,Rogers Park
3,3247427,PERMIT - RENOVATION/ALTERATION,23714.0,Rogers Park
4,3247029,PERMIT - EASY PERMIT PROCESS,23714.0,Rogers Park
...,...,...,...,...
5827,3243364,PERMIT - EASY PERMIT PROCESS,33364.0,Edgewater
5828,3244597,PERMIT - ELECTRIC WIRING,33364.0,Edgewater
5829,3247085,PERMIT - EASY PERMIT PROCESS,33364.0,Edgewater
5830,3246728,PERMIT - EASY PERMIT PROCESS,33364.0,Edgewater


## report

In [7]:
rp_loan = pro_df_2[pro_df_2["permit_type"] == "PERMIT - NEW CONSTRUCTION"]
rp_loan = rp_loan[rp_loan["per_capita_income"] < 30000]
rp_loan = rp_loan.reset_index(drop=True)
rp_loan

,permit_id,permit_type,per_capita_income,community
0,3229248,PERMIT - NEW CONSTRUCTION,21375.0,West Ridge
1,3229248,PERMIT - NEW CONSTRUCTION,21375.0,West Ridge
2,3229248,PERMIT - NEW CONSTRUCTION,21375.0,West Ridge
3,3229248,PERMIT - NEW CONSTRUCTION,21375.0,West Ridge
4,3229248,PERMIT - NEW CONSTRUCTION,24941.0,North Park
5,3229248,PERMIT - NEW CONSTRUCTION,24941.0,North Park
6,3216461,PERMIT - NEW CONSTRUCTION,24941.0,North Park
7,3216461,PERMIT - NEW CONSTRUCTION,24941.0,North Park
8,3216461,PERMIT - NEW CONSTRUCTION,20355.0,Albany Park
9,3216461,PERMIT - NEW CONSTRUCTION,20355.0,Albany Park
